PySpark SQL 零基礎入門教學 🎈
=================================================

這份教學會用最簡單的方式，教你如何使用 PySpark 來學習 SQL。
想像 SQL 就像是和資料庫說話的魔法語言！✨

作者：QChoice AI 教學團隊
日期：2025-01-15

## 🎯 環境設定 - 準備開發環境

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 創建 Spark Session - 就像打開一個魔法工作台
spark = SparkSession.builder \
    .appName("SQL入門教學") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

print("🎉 歡迎學習 SQL！讓我們開始吧！")
print("=" * 60)

## 📚 第一章：基礎查詢操作

## 💪 練習題 1

**題目：**

請建立一個包含書籍資料的 DataFrame（包含：書名、作者、價格、出版年份），並使用 SELECT 查詢所有書籍的書名和價格

**提示：使用 createDataFrame 和 select 方法**

<details>
<summary>📝 點擊查看參考答案</summary>

```python
# 建立書籍資料
books_data = [
    ("Python 入門", "張三", 450, 2023),
    ("資料科學實戰", "李四", 680, 2024),
    ("SQL 精通", "王五", 520, 2023),
    ("機器學習", "趙六", 750, 2024)
]

books_df = spark.createDataFrame(books_data, ["書名", "作者", "價格", "出版年份"])

# 方法 1: DataFrame API
books_df.select("書名", "價格").show()

# 方法 2: SQL 語法
books_df.createOrReplaceTempView("books")
spark.sql("SELECT 書名, 價格 FROM books").show()
```

</details>

---


## 💪 練習題 2

**題目：**

請在書籍資料表中新增 2 本新書

**提示：建立新的 DataFrame 並使用 union 合併**

<details>
<summary>📝 點擊查看參考答案</summary>

```python
# 新書資料
new_books = [
    ("深度學習", "錢七", 820, 2024),
    ("大數據分析", "孫八", 590, 2023)
]

new_books_df = spark.createDataFrame(new_books, ["書名", "作者", "價格", "出版年份"])

# 合併資料
books_df = books_df.union(new_books_df)
books_df.show()

print(f"總書籍數: {books_df.count()}")
```

</details>

---


## 💪 練習題 3

**題目：**

請篩選出價格大於 600 元且出版年份為 2024 年的書籍

**提示：使用 filter 或 where 方法，並使用多個條件**

<details>
<summary>📝 點擊查看參考答案</summary>

```python
# 方法 1: DataFrame API
expensive_new_books = books_df.filter(
    (col("價格") > 600) & (col("出版年份") == 2024)
)
expensive_new_books.show()

# 方法 2: SQL 語法
books_df.createOrReplaceTempView("books")
spark.sql("""
    SELECT * FROM books
    WHERE 價格 > 600 AND 出版年份 = 2024
    ORDER BY 價格 DESC
""").show()
```

</details>

---


In [ ]:
# 1️⃣ SELECT - 查詢資料（從資料表中取出你要的資料）

🎈 概念解釋：
SELECT 就像是你告訴媽媽「我想要看我的玩具」
你可以選擇看全部的玩具，或只看特定的玩具

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，從玩具資料表中選取所有玩具的名稱和顏色，
並顯示前5筆資料。

In [ ]:
# 範例資料：建立一個玩具清單
toys_data = [
    (1, "小熊", "棕色", 299),
    (2, "機器人", "藍色", 599),
    (3, "芭比娃娃", "粉色", 399),
    (4, "樂高", "彩色", 899),
    (5, "小汽車", "紅色", 199)
]

toys_df = spark.createDataFrame(toys_data, ["id", "名稱", "顏色", "價格"])

# SQL 方式 1：使用 SQL 語法
toys_df.createOrReplaceTempView("toys")
result = spark.sql("SELECT `名稱`, `顏色` FROM toys")
print("\n📌 範例 1: SELECT - 選取玩具名稱和顏色")
result.show()

# PySpark DataFrame 方式（另一種寫法）
result2 = toys_df.select("名稱", "顏色")
print("📌 使用 DataFrame API 的結果：")
result2.show()


# 2️⃣ INSERT - 新增資料（在資料表中加入新的記錄）

🎈 概念解釋：
INSERT 就像買了新玩具後，把它放進你的玩具箱
資料庫會記住這個新玩具

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，將一個新玩具「泰迪熊」新增到玩具資料表中，
顏色是「白色」，價格是 450 元。

In [ ]:
# 新玩具資料
new_toy = [(6, "泰迪熊", "白色", 450)]
new_toy_df = spark.createDataFrame(new_toy, ["id", "名稱", "顏色", "價格"])

# 合併資料（模擬 INSERT）
toys_df = toys_df.union(new_toy_df)
toys_df.createOrReplaceTempView("toys")

print("\n📌 範例 2: INSERT - 新增新玩具")
spark.sql("SELECT * FROM toys").show()


# 3️⃣ UPDATE - 更新資料（修改已存在的資料內容）

🎈 概念解釋：
UPDATE 就像你的玩具舊了，你幫它換個新顏色或修理一下
在資料庫裡改變某個資料的內容

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，將玩具「小熊」的價格從 299 元更新為 350 元。

In [ ]:
# PySpark 沒有直接的 UPDATE，我們用 withColumn 和 when
print("\n📌 範例 3: UPDATE - 更新小熊的價格")

toys_df = toys_df.withColumn(
    "價格",
    when(col("名稱") == "小熊", 350).otherwise(col("價格"))
)
toys_df.createOrReplaceTempView("toys")
spark.sql("SELECT * FROM toys WHERE `名稱` = '小熊'").show()


# 4️⃣ DELETE - 刪除資料（移除不需要的記錄）

🎈 概念解釋：
DELETE 就像你決定把壞掉的玩具丟掉
從資料庫移除不需要的資料

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，刪除價格低於 200 元的所有玩具。

In [ ]:
print("\n📌 範例 4: DELETE - 刪除便宜的玩具")

# 使用 filter 保留價格 >= 200 的玩具
toys_df = toys_df.filter(col("價格") >= 200)
toys_df.createOrReplaceTempView("toys")
spark.sql("SELECT * FROM toys").show()

## 📚 第二章：資料庫和資料表管理

## 💪 練習題 4

**題目：**

請建立一個「員工」資料表，包含：員工編號、姓名、部門、薪資、入職日期

**提示：使用 StructType 定義 schema**

<details>
<summary>📝 點擊查看參考答案</summary>

```python
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from datetime import date

# 定義 schema
employee_schema = StructType([
    StructField("員工編號", StringType(), False),
    StructField("姓名", StringType(), False),
    StructField("部門", StringType(), True),
    StructField("薪資", IntegerType(), True),
    StructField("入職日期", DateType(), True)
])

# 建立測試資料
employees_data = [
    ("E001", "張三", "工程部", 55000, date(2022, 1, 15)),
    ("E002", "李四", "業務部", 48000, date(2023, 3, 20)),
    ("E003", "王五", "工程部", 62000, date(2021, 6, 10))
]

# 建立 DataFrame
employees_df = spark.createDataFrame(employees_data, employee_schema)
employees_df.createOrReplaceTempView("employees")
employees_df.show()
```

</details>

---


## 💪 練習題 5

**題目：**

請為員工資料表新增一個「績效等級」欄位，並根據薪資自動分級（>=60000為A級，>=50000為B級，其他為C級）

**提示：使用 withColumn 和 when 函數**

<details>
<summary>📝 點擊查看參考答案</summary>

```python
from pyspark.sql.functions import when, col

# 新增績效等級欄位
employees_df = employees_df.withColumn(
    "績效等級",
    when(col("薪資") >= 60000, "A級")
    .when(col("薪資") >= 50000, "B級")
    .otherwise("C級")
)

employees_df.show()

# 統計各等級人數
employees_df.groupBy("績效等級").count().show()
```

</details>

---


In [ ]:
# 5️⃣ CREATE DATABASE - 建立資料庫（蓋一個新的玩具倉庫）

🎈 概念解釋：
CREATE DATABASE 就像蓋一個全新的倉庫來放你的東西
一個大倉庫裡可以有很多小盒子（資料表）

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，建立一個名為「toy_warehouse」的資料庫。

In [ ]:
print("\n📌 範例 5: CREATE DATABASE - 建立資料庫（概念說明）")

# 在 PySpark 中，我們不需要建立資料庫
# 直接使用臨時視圖即可
print("""
在標準 SQL 中：
  CREATE DATABASE toy_warehouse

在 PySpark 中：
  使用臨時視圖來組織資料，不需要明確建立資料庫
  臨時視圖會自動在 'default' 命名空間中
  
如果需要資料庫功能，可以：
  1. 使用 Hive：設定 spark.sql.catalogImplementation=hive
  2. 使用 Delta Lake：提供更強大的資料管理功能
  3. 使用檔案系統：直接儲存為 Parquet/CSV 等格式
""")

🎈 概念解釋：
CREATE TABLE 就像準備一個有格子的盒子
每個格子都有名字，知道要放什麼東西

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，建立一個「學生成績」資料表，
包含學生姓名、科目、分數三個欄位。

In [ ]:
print("\n📌 範例 6: CREATE TABLE - 建立資料表（使用 DataFrame）")

# 方法 1：使用 DataFrame 定義結構並建立臨時視圖
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

students_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("subject", StringType(), True),
    StructField("score", IntegerType(), True)
])

# 建立空的 DataFrame 作為「資料表」
students_df = spark.createDataFrame([], students_schema)
students_df.createOrReplaceTempView("students")

print("✅ 已建立 students 臨時視圖（類似資料表）")
print("資料表結構：")
students_df.printSchema()

🎈 概念解釋：
DROP DATABASE 會刪除整個資料庫及其所有內容
裡面的所有資料表都會被刪除，使用時需特別小心

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，刪除名為「old_warehouse」的資料庫。
（注意：會刪除所有裡面的資料表）

In [ ]:
print("\n📌 範例 7: DROP DATABASE - 刪除資料庫（概念說明）")

# 在 PySpark 中，我們使用臨時視圖而非資料庫
# 概念說明：
print("""
在標準 SQL 中：
  DROP DATABASE IF EXISTS old_warehouse

在 PySpark 中：
  使用臨時視圖（Temp Views）來組織資料
  可以透過命名規則來模擬資料庫分組，例如：
  - db1_table1
  - db1_table2
  - db2_table1
""")

🎈 概念解釋：
DROP TABLE 會刪除整個資料表
資料庫仍然存在，只是移除了這個資料表

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，刪除「舊玩具」資料表。

In [ ]:
print("\n📌 範例 8: DROP TABLE - 刪除資料表（使用 dropTempView）")

# 先建立一個測試用的臨時視圖
test_data = [(1, "舊玩具")]
test_df = spark.createDataFrame(test_data, ["id", "name"])
test_df.createOrReplaceTempView("old_toys")
print("✅ 已建立 old_toys 臨時視圖")

# 刪除臨時視圖
spark.catalog.dropTempView("old_toys")
print("✅ 已刪除 old_toys 臨時視圖")

# 驗證是否已刪除
try:
    spark.sql("SELECT * FROM old_toys").show()
except Exception as e:
    print(f"✅ 確認已刪除：{type(e).__name__}")

🎈 概念解釋：
ALTER TABLE 可以新增欄位、修改欄位名稱或改變欄位型態
常用於調整資料表結構

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，在玩具資料表中新增一個「製造商」欄位。

In [ ]:
print("\n📌 範例 9: ALTER TABLE - 修改資料表結構")

# 新增欄位
toys_df = toys_df.withColumn("製造商", lit("快樂玩具公司"))
toys_df.createOrReplaceTempView("toys")

spark.sql("SELECT * FROM toys LIMIT 3").show()


# 🔟 TRUNCATE TABLE - 清空資料表（把盒子裡的東西全倒掉）

🎈 概念解釋：
TRUNCATE TABLE 會清空資料表中的所有資料
但資料表結構保留，只是沒有任何記錄

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，清空「暫存資料」資料表的所有內容。

In [ ]:
print("\n📌 範例 10: TRUNCATE TABLE - 清空資料表")

# 建立空的 DataFrame（模擬 TRUNCATE）
empty_df = spark.createDataFrame([], toys_df.schema)
print("✅ 資料表已清空（保留結構）")
empty_df.printSchema()

## 📚 第三章：索引管理

In [ ]:
# 1️⃣1️⃣ CREATE INDEX - 建立索引（做一本目錄）

🎈 概念解釋：
CREATE INDEX 建立索引可以加快查詢速度
想找東西的時候可以更快找到

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，在玩具資料表的「名稱」欄位上建立索引，
以加快查詢速度。（說明：PySpark 使用 cache 來優化）

In [ ]:
print("\n📌 範例 11: CREATE INDEX - 優化查詢速度")

# PySpark 使用 cache() 來優化常用查詢
toys_df.cache()
print("✅ 資料已快取，查詢會更快！")


# 1️⃣2️⃣ DROP INDEX - 刪除索引（把目錄撕掉）

🎈 概念解釋：
DROP INDEX 刪除不需要的索引
不會影響內容，只是找東西會慢一點

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，移除玩具資料表上的快取索引。

In [ ]:
print("\n📌 範例 12: DROP INDEX - 移除快取")

toys_df.unpersist()
print("✅ 快取已移除")

## 📚 第四章：資料表連接（JOIN）

## 💪 練習題 6

**題目：**

請建立「訂單」和「客戶」兩個資料表，然後使用 INNER JOIN 查詢所有訂單及其客戶資訊

**提示：建立兩個 DataFrame，然後使用 join 方法**

<details>
<summary>📝 點擊查看參考答案</summary>

```python
# 建立客戶資料
customers_data = [
    (1, "王小明", "台北市"),
    (2, "李小華", "台中市"),
    (3, "張小美", "高雄市")
]
customers_df = spark.createDataFrame(customers_data, ["客戶編號", "客戶姓名", "城市"])

# 建立訂單資料
orders_data = [
    (101, 1, "2024-01-15", 1500),
    (102, 2, "2024-01-16", 2300),
    (103, 1, "2024-01-17", 890),
    (104, 3, "2024-01-18", 3200)
]
orders_df = spark.createDataFrame(orders_data, ["訂單編號", "客戶編號", "訂單日期", "金額"])

# INNER JOIN
result = orders_df.join(
    customers_df,
    orders_df["客戶編號"] == customers_df["客戶編號"],
    "inner"
).select(
    orders_df["訂單編號"],
    orders_df["訂單日期"],
    customers_df["客戶姓名"],
    customers_df["城市"],
    orders_df["金額"]
)

result.show()

# 計算每個客戶的總消費
customers_df.createOrReplaceTempView("customers")
orders_df.createOrReplaceTempView("orders")

spark.sql("""
    SELECT 
        c.客戶姓名,
        c.城市,
        COUNT(o.訂單編號) as 訂單數,
        SUM(o.金額) as 總消費
    FROM customers c
    INNER JOIN orders o ON c.客戶編號 = o.客戶編號
    GROUP BY c.客戶姓名, c.城市
    ORDER BY 總消費 DESC
""").show()
```

</details>

---


In [ ]:
# 準備範例資料：學生和成績
students_data = [
    (1, "小明", 10),
    (2, "小華", 10),
    (3, "小美", 11)
]

scores_data = [
    (1, "數學", 85),
    (2, "數學", 90),
    (3, "數學", 88),
    (4, "數學", 92)  # 這個學生不在學生表中
]

students_df = spark.createDataFrame(students_data, ["id", "姓名", "年齡"])
scores_df = spark.createDataFrame(scores_data, ["student_id", "科目", "分數"])

students_df.createOrReplaceTempView("students")
scores_df.createOrReplaceTempView("scores")


# 1️⃣3️⃣ INNER JOIN - 內部連接（只拿兩邊都有的）

🎈 概念解釋：
INNER JOIN 只保留兩個資料表都有匹配的記錄
只有兩邊都有的才會被選出來

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，將學生資料表和成績資料表做 INNER JOIN，
只顯示有成績記錄的學生資訊。

In [ ]:
print("\n📌 範例 13: INNER JOIN - 兩邊都有的才留下")

SELECT s.姓名, sc.科目, sc.分數
    FROM students s
    INNER JOIN scores sc ON s.id = sc.student_id

In [ ]:
result.show()


# 1️⃣4️⃣ LEFT JOIN - 左連接（保留左邊全部）

🎈 概念解釋：
LEFT JOIN 保留左表的所有記錄
左邊的全部保留，右邊有配對的就加上，沒有就留空

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，顯示所有學生，即使他們沒有成績記錄也要顯示。

In [ ]:
print("\n📌 範例 14: LEFT JOIN - 保留所有學生")

SELECT s.姓名, sc.科目, sc.分數
    FROM students s
    LEFT JOIN scores sc ON s.id = sc.student_id

In [ ]:
result.show()


# 1️⃣5️⃣ RIGHT JOIN - 右連接（保留右邊全部）

🎈 概念解釋：
RIGHT JOIN 保留右表的所有記錄
右邊的全部保留，左邊有配對的就加上

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，顯示所有成績記錄，
即使某些成績找不到對應的學生也要顯示。

In [ ]:
print("\n📌 範例 15: RIGHT JOIN - 保留所有成績")

SELECT s.姓名, sc.科目, sc.分數
    FROM students s
    RIGHT JOIN scores sc ON s.id = sc.student_id

In [ ]:
result.show()


# 1️⃣6️⃣ FULL OUTER JOIN - 全外部連接（兩邊都保留）

🎈 概念解釋：
FULL OUTER JOIN 保留兩個資料表的所有記錄
不管有沒有配對，全部都要！

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，顯示所有學生和所有成績記錄，
不管是否有配對都要顯示。

In [ ]:
print("\n📌 範例 16: FULL OUTER JOIN - 全部都要")

SELECT s.姓名, sc.科目, sc.分數
    FROM students s
    FULL OUTER JOIN scores sc ON s.id = sc.student_id

In [ ]:
result.show()

## 📚 第五章：集合操作

In [ ]:
# 準備範例資料
class_a = [(1, "小明"), (2, "小華"), (3, "小美")]
class_b = [(3, "小美"), (4, "小強"), (5, "小芳")]

class_a_df = spark.createDataFrame(class_a, ["id", "姓名"])
class_b_df = spark.createDataFrame(class_b, ["id", "姓名"])


# 1️⃣7️⃣ UNION - 聯集（把兩堆玩具合在一起，重複的只留一個）

🎈 概念解釋：
UNION 合併兩個查詢結果
如果有重複的人，只會記錄一次

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，合併 A 班和 B 班的學生名單，
重複的學生只顯示一次。

In [ ]:
print("\n📌 範例 17: UNION - 合併並去除重複")

class_a_df.createOrReplaceTempView("class_a")
class_b_df.createOrReplaceTempView("class_b")

SELECT * FROM class_a
    UNION
    SELECT * FROM class_b

In [ ]:
result.show()


# 1️⃣8️⃣ UNION ALL - 全部聯集（重複的也保留）

🎈 概念解釋：
UNION ALL 合併所有查詢結果
就算有一樣的也不管，全部都要

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，合併兩個班級的點名記錄，
包含所有重複的記錄。

In [ ]:
print("\n📌 範例 18: UNION ALL - 全部合併（含重複）")

SELECT * FROM class_a
    UNION ALL
    SELECT * FROM class_b

In [ ]:
result.show()

## 📚 第六章：資料篩選與排序

In [ ]:
# 1️⃣9️⃣ DISTINCT - 去除重複（只留不一樣的）

🎈 概念解釋：
DISTINCT 去除重複的記錄
重複的口味只拿一顆

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出玩具資料表中所有不同的顏色。

In [ ]:
print("\n📌 範例 19: DISTINCT - 找出所有不同的顏色")

result = spark.sql("SELECT DISTINCT `顏色` FROM toys")
result.show()


# 2️⃣0️⃣ WHERE - 條件篩選（只要符合條件的）

🎈 概念解釋：
WHERE 用來篩選符合條件的資料
設定條件，只有符合的才能被選出來

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出價格大於 400 元的所有玩具。

In [ ]:
print("\n📌 範例 20: WHERE - 篩選貴的玩具")

result = spark.sql("SELECT * FROM toys WHERE `價格` > 400")
result.show()


# 2️⃣1️⃣ ORDER BY - 排序（把東西排排站）

🎈 概念解釋：
ORDER BY 用來排序查詢結果
讓資料有順序

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，將所有玩具按照價格從高到低排序。

In [ ]:
print("\n📌 範例 21: ORDER BY - 價格排序")

result = spark.sql("SELECT * FROM toys ORDER BY `價格` DESC")
result.show()


# 2️⃣2️⃣ GROUP BY - 分組（把同類的放一起）

🎈 概念解釋：
GROUP BY 將資料依指定欄位分組
紅色的放一堆，藍色的放一堆

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，計算每種顏色有幾個玩具。

In [ ]:
print("\n📌 範例 22: GROUP BY - 依顏色分組計數")

SELECT 顏色, COUNT(*) as 數量
    FROM toys
    GROUP BY 顏色

In [ ]:
result.show()


# 2️⃣3️⃣ HAVING - 分組後篩選（分好組後再挑）

🎈 概念解釋：
HAVING 用來篩選分組後的結果
先分組，再選出符合條件的組

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出玩具數量大於 1 個的顏色。

In [ ]:
print("\n📌 範例 23: HAVING - 篩選玩具數量多的顏色")

SELECT 顏色, COUNT(*) as 數量
    FROM toys
    GROUP BY 顏色
    HAVING COUNT(*) > 1

In [ ]:
result.show()

## 📚 第七章：聚合函數（統計運算）

In [ ]:
# 2️⃣4️⃣ COUNT - 計數（數數看有幾個）

🎈 概念解釋：
COUNT 計算記錄的數量
告訴你總共有幾個

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，計算玩具資料表中總共有幾個玩具。

In [ ]:
print("\n📌 範例 24: COUNT - 數玩具總數")

result = spark.sql("SELECT COUNT(*) as `玩具總數` FROM toys")
result.show()


# 2️⃣5️⃣ SUM - 總和（對指定欄位進行加總）

🎈 概念解釋：
SUM 計算數值欄位的總和
算算看總共要花多少錢

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，計算所有玩具的總價值。

In [ ]:
print("\n📌 範例 25: SUM - 計算總價值")

result = spark.sql("SELECT SUM(`價格`) as `總價值` FROM toys")
result.show()


# 2️⃣6️⃣ AVG - 平均（算平均值）

🎈 概念解釋：
AVG 計算數值欄位的平均值
所有價格加起來除以數量

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，計算玩具的平均價格。

In [ ]:
print("\n📌 範例 26: AVG - 計算平均價格")

result = spark.sql("SELECT AVG(`價格`) as `平均價格` FROM toys")
result.show()


# 2️⃣7️⃣ MIN - 最小值（找最小的）

🎈 概念解釋：
MIN 找出最小值
看看哪個價格最低

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出價格最低的玩具價格。

In [ ]:
print("\n📌 範例 27: MIN - 找最低價格")

result = spark.sql("SELECT MIN(`價格`) as `最低價格` FROM toys")
result.show()


# 2️⃣8️⃣ MAX - 最大值（找最大的）

🎈 概念解釋：
MAX 找出最大值
看看哪個價格最高

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出價格最高的玩具及其價格。

In [ ]:
print("\n📌 範例 28: MAX - 找最高價格")

result = spark.sql("SELECT MAX(`價格`) as `最高價格` FROM toys")
result.show()

## 📚 第八章：條件與範圍查詢

In [ ]:
# 2️⃣9️⃣ BETWEEN - 範圍查詢（在某個範圍內）

🎈 概念解釋：
BETWEEN 用來查詢某個範圍內的資料
找出在某個範圍內的東西

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出價格在 300 到 500 元之間的玩具。

In [ ]:
print("\n📌 範例 29: BETWEEN - 找特定價格範圍的玩具")

result = spark.sql("SELECT * FROM toys WHERE `價格` BETWEEN 300 AND 500")
result.show()


# 3️⃣0️⃣ LIKE - 模糊搜尋（名字像什麼的）

🎈 概念解釋：
LIKE 用來進行模糊搜尋
不用完全一樣，有包含就可以

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出名稱中包含「熊」字的所有玩具。

In [ ]:
print("\n📌 範例 30: LIKE - 模糊搜尋玩具名稱")

result = spark.sql("SELECT * FROM toys WHERE `名稱` LIKE '%熊%'")
result.show()


# 3️⃣1️⃣ IN - 清單比對（在這些裡面的）

🎈 概念解釋：
IN 檢查值是否在指定清單中
只要在清單裡的都可以

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出顏色是「紅色」、「藍色」或「粉色」的玩具。

In [ ]:
print("\n📌 範例 31: IN - 找特定顏色的玩具")

result = spark.sql("SELECT * FROM toys WHERE `顏色` IN ('紅色', '藍色', '粉色')")
result.show()


# 3️⃣2️⃣ NOT - 否定（不要這個）

🎈 概念解釋：
NOT 用來反轉條件
把條件相反

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出顏色不是「棕色」的所有玩具。

In [ ]:
print("\n📌 範例 32: NOT - 排除特定條件")

result = spark.sql("SELECT * FROM toys WHERE NOT `顏色` = '棕色'")
result.show()


# 3️⃣3️⃣ IS NULL - 檢查空值（有沒有遺失）

🎈 概念解釋：
IS NULL 檢查欄位是否為空值
找出資料遺失的地方

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出沒有填寫顏色的玩具。

In [ ]:
print("\n📌 範例 33: IS NULL - 找遺失資料")

# 先加一個沒有顏色的玩具
toys_with_null = toys_df.union(
    spark.createDataFrame([(7, "神秘玩具", None, 999, "快樂玩具公司")], 
                         toys_df.schema)
)
toys_with_null.createOrReplaceTempView("toys_with_null")

result = spark.sql("SELECT * FROM toys_with_null WHERE `顏色` IS NULL")
result.show()


# 3️⃣4️⃣ IS NOT NULL - 檢查非空值（有資料的）

🎈 概念解釋：
IS NOT NULL 檢查欄位是否有值
找出有完整資訊的

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出所有有填寫顏色的玩具。

In [ ]:
print("\n📌 範例 34: IS NOT NULL - 找完整資料")

result = spark.sql("SELECT * FROM toys_with_null WHERE `顏色` IS NOT NULL")
result.show()

## 📚 第九章：進階條件邏輯

In [ ]:
# 3️⃣5️⃣ CASE - 條件判斷（如果...那麼...）

🎈 概念解釋：
CASE 根據不同條件回傳不同的值
根據不同情況給不同答案

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，依據價格將玩具分為「昂貴」（>500）、
「中等」（300-500）、「便宜」（<300）三個等級。

In [ ]:
print("\n📌 範例 35: CASE - 價格等級分類")

SELECT 名稱, 價格,
        CASE 
            WHEN 價格 > 500 THEN '昂貴'
            WHEN 價格 >= 300 THEN '中等'
            ELSE '便宜'
        END as 價格等級
    FROM toys

In [ ]:
result.show()


# 3️⃣6️⃣ COALESCE - 取第一個非空值（找替代方案）

🎈 概念解釋：
COALESCE 回傳第一個非 NULL 的值
找第一個不是空的值

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，如果玩具沒有顏色資訊，就顯示「未知」。

In [ ]:
print("\n📌 範例 36: COALESCE - 處理空值")

SELECT 名稱, COALESCE(顏色, '未知') as 顏色
    FROM toys_with_null

In [ ]:
result.show()

## 📚 第十章：子查詢

In [ ]:
# 3️⃣7️⃣ EXISTS - 存在檢查（有沒有這個東西）

🎈 概念解釋：
EXISTS 檢查子查詢是否有結果
檢查是否存在符合條件的資料

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出有昂貴玩具（>500元）的製造商。

In [ ]:
print("\n📌 範例 37: EXISTS - 檢查是否存在")

SELECT DISTINCT 製造商
    FROM toys t1
    WHERE EXISTS (
        SELECT 1 FROM toys t2 
        WHERE t2.製造商 = t1.製造商 AND t2.價格 > 500
    )

In [ ]:
result.show()


# 3️⃣8️⃣ ANY/SOME - 任一比對（只要有一個符合）

🎈 概念解釋：
ANY/SOME 與子查詢的任一值比較
和一堆值比較，有一個符合就可以

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出價格比任一藍色玩具還貴的玩具。

In [ ]:
print("\n📌 範例 38: ANY/SOME - 任一比對")

SELECT 名稱, 價格
    FROM toys
    WHERE 價格 > ANY (SELECT 價格 FROM toys WHERE 顏色 = '藍色')

In [ ]:
result.show()


# 3️⃣9️⃣ ALL - 全部比對（要全部都符合）

🎈 概念解釋：
ALL 與子查詢的所有值比較
和一堆值比較，全部都要符合

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，找出價格比所有藍色玩具都貴的玩具。

In [ ]:
print("\n📌 範例 39: ALL - 全部比對")

SELECT 名稱, 價格
    FROM toys
    WHERE 價格 > ALL (SELECT 價格 FROM toys WHERE 顏色 = '藍色')

In [ ]:
result.show()


# 4️⃣0️⃣ JOIN - 一般連接（把兩個表格接起來）

🎈 概念解釋：
JOIN 將兩個資料表連接起來
找到相同的地方連接起來

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，將玩具表和製造商詳細資料表連接，
顯示玩具名稱和製造商地址。

In [ ]:
print("\n📌 範例 40: JOIN - 連接兩個表格")

# 建立製造商資料
manufacturers = [(1, "快樂玩具公司", "台北市")]
manufacturers_df = spark.createDataFrame(
    manufacturers, ["id", "公司名稱", "地址"]
)
manufacturers_df.createOrReplaceTempView("manufacturers")

SELECT t.名稱, m.公司名稱, m.地址
    FROM toys t
    JOIN manufacturers m ON t.製造商 = m.公司名稱

In [ ]:
result.show()

## 📚 第十一章：資料表約束

In [ ]:
# 4️⃣1️⃣ PRIMARY KEY - 主鍵（每個玩具的身份證）

🎈 概念解釋：
PRIMARY KEY 唯一識別每一筆記錄的欄位
確保每個資料都有獨一無二的識別碼

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，建立一個玩具資料表，
以 id 作為主鍵（唯一識別碼）。

In [ ]:
print("\n📌 範例 41: PRIMARY KEY - 設定主鍵")

CREATE TABLE IF NOT EXISTS toys_pk (
        id INT NOT NULL,
        name STRING,
        PRIMARY KEY (id)
    )

In [ ]:
print("✅ 已建立帶主鍵的資料表")


# 4️⃣2️⃣ FOREIGN KEY - 外鍵（連結到另一個表格）

🎈 概念解釋：
FOREIGN KEY 確保參照的資料存在於另一個資料表中
確保資料之間有正確的關聯

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，建立訂單資料表，
其中客戶 ID 是外鍵，必須參照客戶資料表。

In [ ]:
print("\n📌 範例 42: FOREIGN KEY - 設定外鍵關聯")

# PySpark SQL 語法示範（實際執行可能因版本而異）

SQL 範例：
CREATE TABLE orders (
    order_id INT,
    customer_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
)

In [ ]:
# 4️⃣3️⃣ CONSTRAINT - 約束條件（設定規則）

🎈 概念解釋：
CONSTRAINT 定義資料表的限制條件
給資料設定必須遵守的規則

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，建立玩具資料表，
並設定價格必須大於 0 的約束條件。

In [ ]:
print("\n📌 範例 43: CONSTRAINT - 設定約束條件")

# 使用 DataFrame API 驗證
from pyspark.sql.functions import col

# 驗證價格都大於 0
valid_toys = toys_df.filter(col("價格") > 0)
print(f"✅ 驗證通過，所有玩具價格都大於 0")
valid_toys.select("名稱", "價格").show(3)


# 4️⃣4️⃣ INDEX - 索引優化（加快搜尋速度）

🎈 概念解釋：
INDEX 加快資料查詢的速度
可以快速找到想要的資料

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，對常查詢的欄位建立索引以提升效能。

In [ ]:
print("\n📌 範例 44: INDEX - 效能優化")

# PySpark 使用分區和快取來優化
toys_df.repartition("顏色").cache()
print("✅ 已按顏色分區並快取資料")

## 📚 第十二章：交易控制

In [ ]:
# 4️⃣5️⃣ TRANSACTION - 交易（一起做完或一起取消）

🎈 概念解釋：
TRANSACTION 確保一系列操作要嘛全部成功，要嘛全部失敗
確保多個操作要一起成功或一起失敗

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式說明交易的概念，
將多個操作包在一個交易中執行。

In [ ]:
print("\n📌 範例 45: TRANSACTION - 交易控制")

交易概念說明：
想像你要買 3 個玩具，但錢只夠買 3 個
如果其中一個賣完了，那就全部都不買
這樣才不會只買到一部分玩具

PySpark 中交易通常在寫入資料庫時使用：
- 開始交易 (BEGIN)
- 執行多個操作
- 全部成功就提交 (COMMIT)
- 有錯誤就回滾 (ROLLBACK)

In [ ]:
# 4️⃣6️⃣ COMMIT - 提交（確定要存檔）

🎈 概念解釋：
COMMIT 永久保存所有變更
確定要把改變永久保存

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，示範如何在完成所有操作後提交交易。

In [ ]:
print("\n📌 範例 46: COMMIT - 提交變更")

# 模擬交易提交
try:
    # 執行操作
    new_data = [(8, "新玩具", "黃色", 350, "快樂玩具公司")]
    new_df = spark.createDataFrame(new_data, toys_df.schema)
    
    # 寫入資料（模擬 COMMIT）
    updated_toys = toys_df.union(new_df)
    print("✅ 交易已提交，新玩具已加入")
    updated_toys.select("名稱", "價格").tail(3)
    
except Exception as e:
    print(f"❌ 發生錯誤，交易回滾: {e}")


# 4️⃣7️⃣ ROLLBACK - 回滾（取消不要存檔）

🎈 概念解釋：
ROLLBACK 取消尚未提交的變更
回復到交易開始前的狀態

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，示範當發生錯誤時如何回滾交易。

In [ ]:
print("\n📌 範例 47: ROLLBACK - 回滾交易")

回滾範例：
假設你不小心刪掉了重要資料
ROLLBACK 可以讓你回到刪除之前的狀態
還原所有未提交的操作

在 PySpark 中：
- 可以使用 checkpoint 保存檢查點
- 發生錯誤時可以讀取檢查點資料
- 恢復到之前的狀態

In [ ]:
# 4️⃣8️⃣ SAVEPOINT - 儲存點（設定中途存檔點）

🎈 概念解釋：
SAVEPOINT 在交易中設定中途檢查點
可以選擇回滾到特定的儲存點

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，示範如何在交易中設定多個儲存點。

In [ ]:
print("\n📌 範例 48: SAVEPOINT - 儲存點概念（使用快照替代）")

# 在 PySpark 中，我們使用變數保存不同階段的 DataFrame
# 這類似於設定多個「儲存點」

# 儲存點 1：原始資料
savepoint_1 = toys_df
print("✅ 儲存點 1：原始玩具資料")
print(f"   記錄數：{savepoint_1.count()}")

# 執行一些操作
filtered_df = toys_df.filter(col("價格") > 300)

# 儲存點 2：過濾後的資料
savepoint_2 = filtered_df
print("✅ 儲存點 2：過濾後的資料")
print(f"   記錄數：{savepoint_2.count()}")

# 如果需要回到某個儲存點，直接使用該變數
print("\n💡 回到儲存點 1：")
savepoint_1.show(3)

print("""
在傳統 SQL 中：
  SAVEPOINT sp1;
  -- 執行操作
  SAVEPOINT sp2;
  -- 可以 ROLLBACK TO sp1;

在 PySpark 中：
  使用變數保存不同階段的 DataFrame
  或使用 Delta Lake 的時間旅行功能
""")

🎈 概念解釋：
GRANT 授予使用者特定的操作權限
控制誰可以存取資料

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，授予使用者查詢玩具資料表的權限。

In [ ]:
print("\n📌 範例 49: GRANT - 授予權限")

權限授予範例：

-- 讓 user1 可以查詢玩具表
GRANT SELECT ON toys TO user1;

-- 讓 user2 可以新增玩具
GRANT INSERT ON toys TO user2;

-- 讓 admin 擁有所有權限
GRANT ALL ON toys TO admin;

In [ ]:
# 5️⃣0️⃣ REVOKE - 撤銷權限（收回鑰匙）

🎈 概念解釋：
REVOKE 撤銷使用者的權限
移除已授予的存取權限

🎯 AI Prompt 範例：
請幫我寫一個 PySpark 程式，撤銷使用者對玩具資料表的刪除權限。

In [ ]:
print("\n📌 範例 50: REVOKE - 撤銷權限")

權限撤銷範例：

-- 不讓 user1 刪除玩具
REVOKE DELETE ON toys FROM user1;

-- 收回 user2 的所有權限
REVOKE ALL ON toys FROM user2;

In [ ]:
# 🎓 課程總結

print("\n" + "=" * 60)
print("🎊 恭喜！你已經完成了 SQL 的 50 個重要觀念！")
print("=" * 60)

📝 學習總結：

1️⃣ 基礎操作：SELECT, INSERT, UPDATE, DELETE
   - 這些是最常用的基本操作

2️⃣ 資料表管理：CREATE, DROP, ALTER, TRUNCATE
   - 管理資料庫結構

3️⃣ 連接操作：INNER JOIN, LEFT JOIN, RIGHT JOIN, FULL OUTER JOIN
   - 連接多個資料表

4️⃣ 資料篩選：WHERE, HAVING, DISTINCT, ORDER BY, GROUP BY
   - 篩選和排序資料

5️⃣ 聚合函數：COUNT, SUM, AVG, MIN, MAX
   - 執行統計運算

6️⃣ 進階查詢：CASE, EXISTS, ANY, ALL, BETWEEN, LIKE, IN
   - 進階的查詢技巧

7️⃣ 交易控制：TRANSACTION, COMMIT, ROLLBACK, SAVEPOINT
   - 維護資料一致性

8️⃣ 權限管理：GRANT, REVOKE
   - 管理使用者權限

💡 學習建議：
- 每個概念都要實際練習
- 嘗試用真實的資料練習
- 遇到問題可以尋求協助
- 循序漸進地學習

🚀 下一步：
- 練習組合多個 SQL 語句
- 應用於實際案例
- 深入學習進階功能

記住：熟能生巧，多練習就能掌握 SQL！

In [ ]:
print("=" * 60)
print("📚 教學文件結束 - 祝你學習愉快！🎈")
print("=" * 60)

# 關閉 Spark Session
# spark.stop()  # 取消註解以關閉 Spark